In [162]:
!pip install langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 7.8 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.8 MB/s eta 0:00:0031m12.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.8 MB/s eta 0:00:0031m13.9 MB/s eta 0:00:01
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp39-cp39-macosx_11_0_arm64.whl (29 kB)
 

In [3]:
!pip install langchain sentence_transformers

import os
import pandas as pd
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/Users/fdurrani/LocalGitHub/legislation_llm/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 876kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 245kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 20.6MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 241kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 49.0MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:06<00:00, 15.0MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 237kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 589kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 11.3MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 1.78MB/s]

In [163]:
def get_files(dir_path: list) -> dict:
    files = [file for file in os.listdir(dir_path)]
    file_paths = [dir_path + file for file in files]
    return pd.DataFrame({"file_name": files, "file_path": file_paths})


def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()


def get_text(db: pd.DataFrame) -> pd.DataFrame:
    db["text"] = db["file_path"].apply(read_file)
    return db


# def get_context(db: pd.DataFrame, overlap: int = 300) -> pd.DataFrame:
#     context = []
#     for row in db.iterrows():
#         i = row["chunk"].metadata["start_index"]
#         if i - overlap < 0 or i + overlap > len(row["text"]):
#             continue

#         context.append(row["text"][i - overlap : i + overlap])

#     db["context"] = context
#     return db


def get_context_for_row(row, chunk_size, overlap):
    start_index = row["chunk"].metadata["start_index"]
    text_length = len(row["text"])

    if start_index - overlap < 0:
        return row["text"][start_index : start_index + chunk_size + overlap]
    if start_index + chunk_size + overlap > text_length:
        return row["text"][start_index - overlap : text_length]

    return row["text"][start_index - overlap : start_index + chunk_size + overlap]


def get_context(db: pd.DataFrame, chunk_size, overlap: int = 100) -> pd.DataFrame:
    # Apply the function to each row of the DataFrame
    db["context"] = db.apply(
        get_context_for_row, chunk_size=chunk_size, overlap=overlap, axis=1
    )
    return db


def get_chunks(db: pd.DataFrame, text_splitter) -> pd.DataFrame:
    db["chunk"] = db["text"].apply(lambda s: text_splitter.create_documents([s]))
    return db.explode("chunk")


def get_embeddings(db: pd.DataFrame, model) -> pd.DataFrame:
    db["embedding"] = db["chunk"].apply(lambda s: model.encode(s.page_content))
    return db


def separate_tables(db):
    return db[["file_name", "file_path", "text"]].drop_duplicates(
        subset="file_name"
    ), db.drop(columns=['file_path', 'text'])


def to_postgres():
    NotImplemented


CHUNK_SIZE = 512

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)
db = get_files(os.path.join(os.getcwd(), "data/"))
db = get_text(db)
db = get_chunks(db, text_splitter)
db = get_context(db, CHUNK_SIZE)
db = get_embeddings(db, model)
text_db, vector_db = separate_tables(db)
#to_postgres(vector_db)

In [164]:
text_db

,file_name,file_path,text
0,2226.txt,c:\Users\nicho\Documents\legislation_llm\pipel...,To authorize appropriations for fiscal year 2...
1,3076.txt,c:\Users\nicho\Documents\legislation_llm\pipel...,[Congressional Bills 117th Congress]\n[From th...


In [165]:
vector_db

,file_name,chunk,context,embedding
0,2226.txt,page_content='To authorize appropriations for ...,To authorize appropriations for fiscal year 20...,"[-0.04015888, 0.0014534396, 0.03527895, -0.018..."
0,2226.txt,"page_content=""(a) Short Title.--This Act may b...",e \nUnited States of America in Congress assem...,"[-0.03192492, -0.020027928, -0.0030170872, -0...."
0,2226.txt,page_content='Subtitle A--Authorization of App...,DEPARTMENT OF DEFENSE AUTHORIZATIONS\n\n ...,"[-0.069968805, 0.0012165786, 0.009229354, -0.0..."
0,2226.txt,page_content='Sec. 111. Report on Army require...,tions\n\nSec. 101. Authorization of appropriat...,"[-0.065443374, 0.06627812, -0.008602901, -0.04..."
0,2226.txt,page_content='augmentation program of the Army...,on plants.\nSec. 115. Report on acquisition st...,"[-0.092744105, -0.031237988, 0.04267691, -0.00..."
...,...,...,...,...
1,3076.txt,"page_content='Regulatory Commission, but such ...",pending on the effective \n date of this s...,"[-0.035138424, 0.07144515, 0.031928845, -0.011..."
1,3076.txt,page_content='(4) Suits.--This section and the...,en discontinued or modified \n if this sect...,"[-0.03640648, 0.09893074, 0.059135232, -0.0388..."
1,3076.txt,page_content='any document relating to the Ins...,"any other Federal law, \n Executive order,...","[-0.008682428, 0.07932113, 0.126732, -0.013632..."
1,3076.txt,page_content='TITLE III--SEVERABILITY\n\n S...,ect on the date that is 180 days after the dat...,"[-0.060060788, -0.008277961, 0.039201688, -0.0..."


In [183]:
# CONFIG

Your instance connection name is: legistlation-llm:us-east4:legislation-llm-vectordb


In [9]:
INSTANCE_CONNECTION_NAME = f"legistlation-llm:us-east4:legislation-llm-vectordb"  # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "postgres"
DB_PASS = "password"
DB_NAME = "postgres"

Your instance connection name is: legistlation-llm:us-east4:legislation-llm-vectordb


In [22]:
!pip install cloud-sql-python-connector sqlalchemy pg8000

from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()


# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME, "pg8000", user=DB_USER, password=DB_PASS, db=DB_NAME
    )
    return conn


# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.2 MB/s eta 0:00:00


In [23]:
pool

Engine(postgresql+pg8000://)

In [26]:
with pool.connect() as db_conn:
    # db_conn.execute(sqlalchemy.text("CREATE TABLE * FROM legislation_vector_db_001"))
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()
    print(results)

[(1, 'HOTDOG', 'Germany', 7.5), (2, 'BÀNH MÌ', 'Vietnam', 9.1), (3, 'CROQUE MADAME', 'France', 8.3), (4, 'CROQUE MADAME', 'France', 8.3)]


In [80]:
TABLE_NAME = "legislation_vector_db_003"

def create_insert_statement():
  insert_list = []
  for row in vector_db.iterrows():
    insert_list.append(
      f"""('{row[1]["file_name"]}',
      '{row[1]["chunk"].page_content.replace("'", "''")}',
      '{row[1]["context"].replace("'", "''")}', 
      '{str(list(row[1]["embedding"]))}')"""
    )
  return f"""INSERT INTO {TABLE_NAME} (file_name, chunk, context, embedding) VALUES {','.join(insert_list)}"""

# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  db_conn.execute(
    sqlalchemy.text(
      f"""DROP TABLE IF EXISTS {TABLE_NAME};"""
    )
  )

  db_conn.execute(
    sqlalchemy.text(
      f"""CREATE TABLE {TABLE_NAME}
      (file_name VARCHAR NOT NULL,
      chunk VARCHAR NOT NULL,
      context VARCHAR NOT NULL,
      embedding VECTOR NOT NULL);"""
    )
  )

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # insert data into our ratings table
  insert_stmt = sqlalchemy.text(
      create_insert_statement(),
  )

  # insert entries into table
  db_conn.execute(insert_stmt)

  # commit transactions
  db_conn.commit()

  # query and fetch ratings table
  results = db_conn.execute(sqlalchemy.text(f"SELECT * FROM {TABLE_NAME}")).fetchall()

  # show results
  print(len(results))
  print(results[0])

225
('3076.txt', '[Congressional Bills 117th Congress]\n[From the U.S. Government Publishing Office]\n[H.R. 3076 Enrolled Bill (ENR)]\n\n        H.R.3076\n\n           ... (217 characters truncated) ... ld at the City of Washington on Monday,\n          the third day of January, two thousand and twenty-two\n\n\n                                 An Act', '[Congressional Bills 117th Congress]\n[From the U.S. Government Publishing Office]\n[H.R. 3076 Enrolled Bill (ENR)]\n\n        H.R.3076\n\n           ... (338 characters truncated) ...                       An Act\n\n\n \n To provide stability to and enhance the services of the United States \n                 Postal Service, and fo', '[-0.014183788,-0.02924115,-0.003241075,-0.008037433,-0.06743431,0.024757728,-0.04988254,-0.051518813,-0.050586935,0.016967984,-0.03050977,0.036682453 ... (4388 characters truncated) ... 111473,0.034941643,0.013506848,0.047369294,0.098187834,0.03133701,-0.06338935,0.032592315,-0.0354372,0.07228599,0.057038